In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from math import exp
import time
from itertools import product

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [3]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
sns.set_style("whitegrid")
sns.set_context("poster")
import pickle

In [4]:
def get_all_movies_via_search(ext):
    driver = webdriver.Firefox()
    driver.maximize_window()

    wait = WebDriverWait(driver, 10)

    url = "http://www.rottentomatoes.com/search/?search="
    driver.get(url+ext)

    # wait for the complete page load
    wait.until(EC.invisibility_of_element_located((By.ID, "preload")))
    
    try:
        element = driver.find_element_by_xpath("//*[@id='movies_tab']")
        element.click()
    except NoSuchElementException:
        driver.close()
        return []
    else:
        wait.until(EC.invisibility_of_element_located((By.ID, "preload")))

        # parse the HTML
        soup = BeautifulSoup(driver.page_source, "html.parser")
        soups = []
        counter = 1
        while True:
            try:
                element = driver.find_element_by_xpath("(//div[@id='results_movies_tab']//span[contains(@class, 'right')])[1]")
                element.click()
            except NoSuchElementException:
                driver.close()
                return [soup]
            else:
                time.sleep(5)
                soup2 = BeautifulSoup(driver.page_source, "html.parser")
                if soup == soup2:
                    soups.append(soup)
                    break
                else:
                    soups.append(soup)
                    soup = soup2
        driver.close()
        return soups

def movies(soup):
    soup_ = soup.find_all("ul", id = "movie_results_ul")[1]
    i = 1
    for s in soup_.find_all("li"):
        print i, s.find_all("a")[1].text
        i += 1

In [5]:
all_movies_soups = {}
strng = "abcdefghijklmnopqrstuvwxyz0123456789"

In [6]:
comb_string = ["".join(a) for a in product(strng, repeat = 2)]

In [ ]:
n = len(comb_string)

In [ ]:
while True:
    try:
        for i, a in enumerate(comb_string):
            if a in all_movies_soups.keys():
                if a == strng[-1]:
                    break
                print "\r", float(i)/n, "Already scraped", a,
            else:
                print "\r", float(i)/n, a,
                all_movies_soups[a] = get_all_movies_via_search(a)
    except KeyboardInterrupt:
        break
    except:
        pass